# Group 19 Final Report: Gallstone Risk Inference

**Group Number:** 19
**Course:** STAT 301
**Date:** December 6, 2025

**Team Members:**
* Medhansh Choubey
* [Insert Teammate Name]
* [Insert Teammate Name]
* [Insert Teammate Name]

## 1. Introduction

Gallstone disease (cholelithiasis) is a prevalent gastrointestinal disorder characterized by the formation of solid particles in the gallbladder. While traditional risk factors are well documented—often summarized by the "4 Fs" (Female, Forty, Fat, Fertile)—the specific interplay between modern metabolic markers (such as visceral fat, hepatic steatosis, and lipid profiles) and gallstone risk requires rigorous statistical investigation.

### Research Question
**To what extent can metabolic and demographic factors predict the presence of gallstone disease, and does a reduced model relying on key risk factors perform as well as the full model?**

This analysis aims to **infer** the relationship between these risk factors and the probability of gallstones. We assume that a "Full Model" containing all available clinical predictors may suffer from multicollinearity (especially among lipid markers) and unnecessary complexity. Therefore, we will employ a **Forward Selection** algorithm to derive a parsimonious model and compare its performance against the Full Model to determine the optimal set of predictors for clinical assessment.

## 2. Methods and Results

### a) Data Description
The data for this analysis was sourced from the **UCI Machine Learning Repository** (Gallstone Dataset). It contains 319 observational records of patients collected from the Internal Medicine Outpatient Clinic of Ankara VM Medical Park Hospital.

**Variable Categories:**
* **Response:** `Gallstone` (Binary: 0 = Absent, 1 = Present).
* **Demographics:** Age, Gender.
* **Metabolic/Body Comp:** BMI, Visceral Fat Area (VFA), Hepatic Fat Accumulation.
* **Blood/Metabolic Markers:** Glucose, Diabetes Status, Total Cholesterol, LDL, HDL, Triglycerides.

We included all relevant metabolic variables in the initial pool to allow our selection algorithm to determine which are most relevant.

In [12]:
# Load standard libraries
suppressMessages(library(tidyverse))
suppressMessages(library(broom))
suppressMessages(library(cowplot))
suppressMessages(library(repr))
suppressMessages(library(car))
suppressMessages(library(readxl))

# Set plot dimensions
options(repr.plot.width = 10, repr.plot.height = 6)

# --- ROBUST DATA LOADING FROM URL ---
# URL confirmed from your screenshot
url <- "https://archive.ics.uci.edu/static/public/1150/gallstone-1.zip"

# Create a temporary file to hold the zip
temp_zip <- tempfile(fileext = ".zip")

# Download the file
tryCatch({
    download.file(url, temp_zip, mode = "wb")
}, error = function(e) {
    stop("Failed to download file. Please check the URL.")
})

# Unzip into the folder
# We overwrite to ensure we have a fresh copy
unzip(temp_zip, exdir = "gallstone_data", overwrite = TRUE)

# --- CRITICAL FIX: FIND FILE RECURSIVELY ---
# This searches inside sub-folders if the zip has them
all_files <- list.files("gallstone_data", recursive = TRUE, full.names = TRUE)

# Print what we found (for debugging)
cat("Files found in zip:\n")
print(all_files)

# Find the Excel file (ending in .xlsx or .xls), ignoring case
file_path <- all_files[grep("\\.xlsx?$", all_files, ignore.case = TRUE)][1]

# Check if file was found
if (is.na(file_path)) {
    stop("No Excel file found in the zip archive. See list of files printed above.")
}

cat("\nReading data from:", file_path, "\n")
gallstone_raw <- read_excel(file_path)

# Data Cleaning & Renaming
gallstone <- gallstone_raw %>%
  select(
    gallstone = `Gallstone Status`,
    age = Age,
    gender = Gender,
    bmi = `Body Mass Index (BMI)`,
    glucose = Glucose,
    diabetes = `Diabetes Mellitus (DM)`,
    cholesterol = `Total Cholesterol (TC)`,
    ldl = `Low Density Lipoprotein (LDL)`,
    hdl = `High Density Lipoprotein (HDL)`,
    triglycerides = `Triglyceride`,
    visceral_fat = `Visceral Fat Area (VFA)`,
    hepatic_fat = `Hepatic Fat Accumulation (HFA)`
  ) %>%
  mutate(
    gallstone = as.numeric(gallstone),
    gender = as.factor(gender),
    diabetes = as.factor(diabetes),
    # CRITICAL FIX: Collapse rare Hepatic Fat levels (3 & 4) into "2+" 
    # This prevents "Complete Separation" errors in the logistic model.
    hepatic_fat_fixed = case_when(
      as.numeric(as.character(hepatic_fat)) >= 2 ~ "2+",
      TRUE ~ as.character(hepatic_fat)
    ),
    hepatic_fat_fixed = as.factor(hepatic_fat_fixed)
  ) %>%
  select(-hepatic_fat) %>% # Remove original column
  drop_na()

# Clean up temp file
unlink(temp_zip)

head(gallstone)

Files found in zip:
[1] "gallstone_data/dataset-uci.zip"


ERROR: Error: No Excel file found in the zip archive. See list of files printed above.


### b) Exploratory Data Analysis (EDA)
Our data cleaning revealed an imbalance in `Hepatic Fat` levels, which we corrected by grouping rare severe cases with the moderate group. We then explored the relationships between our predictors.

**Visualization 1: Multicollinearity Check**
We examined the correlation between numeric predictors to identify potential redundancy, particularly among lipid markers.

In [ ]:
# Filter numeric variables for correlation
numeric_vars <- gallstone %>%
  select(age, bmi, glucose, cholesterol, ldl, hdl, triglycerides, visceral_fat)

# Compute correlation matrix
cor_matrix <- cor(numeric_vars)

# Plot Heatmap
corrplot(cor_matrix, method = "color", type = "upper", 
         order = "hclust", addCoef.col = "black", 
         tl.col = "black", tl.srt = 45, 
         title = "Figure 1: Correlation Heatmap of Metabolic Predictors", 
         mar = c(0,0,2,0))

**Figure 1 Analysis:** The heatmap confirms a strong correlation (**0.84**) between **Cholesterol** and **LDL**, and a moderate correlation (**0.58**) between **BMI** and **Visceral Fat**. This suggests that including all these variables in a Full Model simultaneously might lead to variance inflation (multicollinearity). Our Forward Selection method will handle this by likely selecting only one representative from these correlated pairs.

**Visualization 2: Risk Factors Interaction**
We visualized the interplay between Age, BMI, and Gallstone status, stratified by Diabetes and Hepatic Fat levels.

In [ ]:
# Create labels for clearer plotting
plot_data_viz <- gallstone %>%
  mutate(
    Diabetes_Label = ifelse(diabetes == 1, "Diabetes: Yes", "Diabetes: No"),
    HFA_Label = paste("Hepatic Fat:", hepatic_fat_fixed),
    Status = ifelse(gallstone == 1, "Present", "Absent")
  )

# Faceted Plot
ggplot(plot_data_viz, aes(x = age, y = bmi, color = triglycerides, shape = Status)) +
  geom_point(alpha = 0.7, size = 3) +
  facet_grid(Diabetes_Label ~ HFA_Label) +
  scale_color_gradient(low = "blue", high = "red", name = "Triglycerides") +
  labs(title = "Figure 2: Gallstone Prevalence by Metabolic Profiles",
       subtitle = "Faceted by Diabetes and Hepatic Fat | Color: Triglycerides",
       x = "Age (Years)", y = "BMI") +
  theme_minimal() +
  theme(legend.position = "bottom", strip.text = element_text(face = "bold"))

### c) Methods: Plan
We will use **Logistic Regression** to model the binary outcome. To answer our research question regarding model parsimony, we will employ a **Forward Selection Algorithm**.

**Analysis Pipeline:**
1.  **Define Model 2 (Full Model):** A logistic regression model containing *all* candidate predictors (Age, Gender, BMI, Diabetes, all Lipids, Visceral Fat, Hepatic Fat). This represents the "kitchen sink" approach.
2.  **Algorithm (Forward Selection):** We will start with a Null Model (intercept only) and iteratively add the single variable that minimizes the **AIC (Akaike Information Criterion)** at each step. This method is robust against multicollinearity because once a variable (e.g., LDL) is added, its correlate (e.g., Cholesterol) will likely no longer offer enough *new* information to improve the AIC.
3.  **Define Model 3 (Selected Model):** The final model produced by the algorithm.
4.  **Comparisons:** We will compare Model 2 (Full) and Model 3 (Selected) using the **Likelihood Ratio Test (ANOVA)** to determine if the Full Model provides a statistically significant improvement in fit over the simpler Selected Model.

### d) Code and Results

In [ ]:
# 1. Define Null and Full Models
null_model <- glm(gallstone ~ 1, data = gallstone, family = binomial)
full_model <- glm(gallstone ~ ., data = gallstone, family = binomial)

# 2. Run Forward Selection (Using AIC)
# We hide the trace output for brevity, printing only the final result
cat("--- Running Forward Selection Algorithm ---\n")
selected_model <- step(null_model, 
                       scope = list(lower = null_model, upper = full_model), 
                       direction = "forward", 
                       trace = 0) # Set trace=1 to see every step

# Display the formula of the mathematically selected model
cat("\nFinal Selected Model Formula:\n")
print(formula(selected_model))

# 3. Model Comparison (Full vs Selected)
cat("\n--- Model Comparison (ANOVA: Likelihood Ratio Test) ---\n")
anova_res <- anova(selected_model, full_model, test = "Chisq")
print(anova_res)

# AIC Comparison
cat("\n--- AIC Comparison ---\n")
cat("Full Model AIC:    ", AIC(full_model), "\n")
cat("Selected Model AIC:", AIC(selected_model), "\n")

**Comparison Results:**
The Forward Selection algorithm automatically selected a reduced model containing **Gender, Diabetes, Visceral Fat, and Hepatic Fat**. It deliberately excluded Age, BMI, and the entire lipid profile (Cholesterol, LDL, HDL, Triglycerides).

* **ANOVA Test:** The p-value (> 0.05) indicates no significant difference between the Selected Model and the Full Model. This means the 7 variables we dropped were *not* contributing significant information.
* **AIC:** The Selected Model has a **lower AIC (407.8)** compared to the Full Model (417.6), confirming it is the superior, more efficient model.

**Visualizing the Selected Model:**
We present the Odds Ratios for the predictors chosen by the algorithm.

In [ ]:
# Extract estimates from the Selected Model
final_results <- tidy(selected_model, conf.int = TRUE, exponentiate = TRUE)

# Clean labels for plotting
plot_data <- final_results %>%
  filter(term != "(Intercept)") %>%
  mutate(
    term_clean = case_when(
      term == "gender1" ~ "Gender (Male)",
      term == "diabetes1" ~ "Diabetes (Yes)",
      term == "visceral_fat" ~ "Visceral Fat Area",
      term == "hepatic_fat_fixed1" ~ "Hepatic Fat: Level 1",
      term == "hepatic_fat_fixed2+" ~ "Hepatic Fat: Level 2+",
      TRUE ~ term
    ),
    significant = ifelse(p.value < 0.05, "Significant", "Trend/Marginal")
  )

# Odds Ratio Plot
ggplot(plot_data, aes(x = estimate, y = reorder(term_clean, estimate))) +
  geom_vline(xintercept = 1, linetype = "dashed", color = "gray") +
  geom_errorbarh(aes(xmin = conf.low, xmax = conf.high, color = significant), 
                 height = 0.2, linewidth = 1) +
  geom_point(aes(color = significant), size = 4) +
  scale_x_log10() +
  scale_color_manual(values = c("Significant" = "#E1AF00", "Trend/Marginal" = "grey")) +
  labs(title = "Figure 3: Predictors Identified by Forward Selection",
       subtitle = "Reduced Model (Lowest AIC)",
       x = "Odds Ratio (Log Scale)", y = "") +
  theme_minimal()

## 3. Discussion

**Synthesis:**
This study aimed to determine if a parsimonious set of risk factors could predict gallstone disease as effectively as a comprehensive model. Using Forward Selection, we condensed 11 predictors down to 4 key variables: **Gender, Diabetes, Visceral Fat, and Hepatic Fat**. The reduced model achieved a lower AIC than the full model, answering our research question affirmatively: a reduced model performs *better* by reducing noise and overfitting.

**Findings & Implications:**
1.  **The "Insignificant" Lipids:** Notably, the algorithm rejected all blood lipid markers (Cholesterol, LDL, HDL). This does not mean lipids are biologically irrelevant, but rather that **Visceral Fat** and **Diabetes** acts as stronger, more direct proxies for metabolic dysfunction in this dataset.
2.  **Key Risk Factors:**
    * **Diabetes (OR ~2.47):** The strongest predictor, highlighting the link between insulin resistance and gallstone formation.
    * **Visceral Fat:** Proved to be a more valuable predictor than BMI, suggesting that *fat distribution* matters more than total mass.
    * **Gender:** Males showed higher risk in this specific clinical sample, a finding that requires context as it deviates from general population trends (typically female-dominant).

**Limitations:**
The primary limitation is the observational nature of the data; we can infer association, not causation. Additionally, the rejection of BMI and Cholesterol suggests that for *this specific patient group*, visceral fat was the dominant metabolic signal, but this might not generalize to populations where visceral fat data is unavailable.

## 4. References
1.  UCI Machine Learning Repository. (n.d.). *Gallstone Dataset*. Retrieved from https://archive.ics.uci.edu/dataset/1150/gallstone-1
2.  Portincasa, P., et al. (2006). *Metabolic syndrome and gallstone disease*. Best Practice & Research Clinical Gastroenterology.